# CryptoCompare example

Crypto Currency Arbitrage Trading Back Testing within CryptoCompare.

This document discribed how to get data from CryptoCompare to analyze arbitrage opportunity.

# 1. Calc the common coins in Okex, Binance, Huobi and gate

In [1]:
# get the exchange and coins data from CryptoCompare

import requests

exchangeCoinFile = open("../data/cryptocompare/cryptocompare.exchange.json","w+")

exchangeCoinQuerryURL = "https://min-api.cryptocompare.com/data/all/exchanges"

exchangeCoinResponse = requests.request("GET", exchangeCoinQuerryURL)

exchangeCoinFile.write(exchangeCoinResponse.text)

exchangeCoinFile.close

print("done")

done


In [2]:
# calc the common coins in Okex Binance Huobi Gate

import json

exchangeCoinFile = "../data/cryptocompare/cryptocompare.exchange.json"

# Reading data back
with open(exchangeCoinFile, 'r') as f:
    exchangeCoinData = json.load(f)

okexCoin = exchangeCoinData["OKEX"]
binanceCoin = exchangeCoinData["Binance"]
huobiCoin = exchangeCoinData["HuobiPro"]
gateCoin = exchangeCoinData["Gateio"]

okexBinanceCommonKeys = okexCoin.keys() & binanceCoin.keys()
okexHuobiCommonKeys = okexCoin.keys() & huobiCoin.keys()
okexGateCommonKeys = okexCoin.keys() & gateCoin.keys()
binanceHuobicommonKeys = binanceCoin.keys() & huobiCoin.keys()
binanceGateCommonKeys = binanceCoin.keys() & gateCoin.keys()
huobiGateCommonKeys = huobiCoin.keys() & gateCoin.keys()

okexBinanceCoin = {}
okexHuobiCoin = {}
okexGateCoin = {}
binanceHuobiCoin = {}
binanceGateCoin = {}
huobiGateCoin = {}

for key in okexBinanceCommonKeys:
    okexBinanceCoin[key] = list(set(okexCoin[key]).intersection(set(binanceCoin[key])))
for key in okexHuobiCommonKeys:
    okexHuobiCoin[key] = list(set(okexCoin[key]).intersection(set(huobiCoin[key])))
for key in okexGateCommonKeys:
    okexGateCoin[key] = list(set(okexCoin[key]).intersection(set(gateCoin[key])))
for key in binanceHuobicommonKeys:
    binanceHuobiCoin[key] = list(set(binanceCoin[key]).intersection(set(huobiCoin[key])))
for key in binanceGateCommonKeys:
    binanceGateCoin[key] = list(set(binanceCoin[key]).intersection(set(gateCoin[key])))
for key in huobiGateCommonKeys:
    huobiGateCoin[key] = list(set(huobiCoin[key]).intersection(set(gateCoin[key])))

okexBinanceCoinFile = "../data/cryptocompare/cryptocompare.exchange.okexBinanceCoin.json"
okexHuobiCoinFile = "../data/cryptocompare/cryptocompare.exchange.okexHuobiCoin.json"
okexGateCoinFile = "../data/cryptocompare/cryptocompare.exchange.okexGateCoin.json"
binanceHuobiCoinFile = "../data/cryptocompare/cryptocompare.exchange.binanceHuobiCoin.json"
binanceGateCoinFile = "../data/cryptocompare/cryptocompare.exchange.binanceGateCoin.json"
huobiGateCoinFile = "../data/cryptocompare/cryptocompare.exchange.huobiGateCoin.json"

with open(okexBinanceCoinFile, 'w+') as f:
    json.dump(okexBinanceCoin, f)
with open(okexHuobiCoinFile, 'w+') as f:
    json.dump(okexHuobiCoin, f)
with open(okexGateCoinFile, 'w+') as f:
    json.dump(okexGateCoin, f)
with open(binanceHuobiCoinFile, 'w+') as f:
    json.dump(binanceHuobiCoin, f)
with open(binanceGateCoinFile, 'w+') as f:
    json.dump(binanceGateCoin, f)
with open(huobiGateCoinFile, 'w+') as f:
    json.dump(huobiGateCoin, f)

print("done")

done


In [3]:
# define common function
import sys
import json
import time
import requests
import numpy as np
import pandas as pd

def calcDayArbitrage(fSymbol, tSymbol, xExchange, xFee, yExchange, yFee, timeStamp):
    # handle timestamp (senconds to hour)
    timeStamp = timeStamp - timeStamp%1440
    # cryptoCompare api
    url = "https://min-api.cryptocompare.com/data/histominute"
    # querry str
    xQuerryStr =  {"fsym":fSymbol,"tsym":tSymbol,"limit":"1440","aggregate":"1",  
               "e":xExchange,"toTs":timeStamp}
    yQuerryStr =  {"fsym":fSymbol,"tsym":tSymbol,"limit":"1440","aggregate":"1",  
               "e":yExchange,"toTs":timeStamp}
    # request response
    try:
        xResponse = requests.request("GET", url, params=xQuerryStr)
        yResponse = requests.request("GET", url, params=yQuerryStr)
        if xResponse.status_code == requests.codes.ok and yResponse.status_code == requests.codes.ok:
            # get xDataFrame, yDataFrame
            xDataFrame = pd.DataFrame(xResponse.json()["Data"])
            yDataFrame = pd.DataFrame(yResponse.json()["Data"])
            # calc measurements
            measurements = calcMeasurements(xDataFrame, yDataFrame, xFee, yFee)
            chance = 0
            profit = 0
            volume = 0
            candy = 0
            for index in range(len(measurements)):
                if measurements.loc[index,"profit"] > 0:
                    if measurements.loc[index,"difference"] != 0 and candy == 0:
                        if measurements.loc[index,"difference"] > 0:
                            candy = 1
                        else:
                            candy = -1
                        chance = 1
                        profit = measurements.loc[index,"profit"]
                        volume = measurements.loc[index,"volume"]
                    if measurements.loc[index,"difference"] > 0 and candy < 0:
                        candy = 1
                        chance = chance + 1
                        profit = profit + measurements.loc[index,"profit"]
                        volume = volume + measurements.loc[index,"volume"]
                    if measurements.loc[index,"difference"] < 0 and candy > 0:
                        candy = -1
                        chance = chance + 1
                        profit = profit + measurements.loc[index,"profit"]
                        volume = volume + measurements.loc[index,"volume"]
            return [chance, profit, volume]
        else:
            return [-1, -1, -1]
    except requests.exceptions.RequestException as e:
        print(e)
        sys.exit(1)

def calcHourArbitrage(fSymbol, tSymbol, xExchange, xFee, yExchange, yFee, timeStamp):
    # handle timestamp (senconds to hour)
    timeStamp = timeStamp - timeStamp%60
    # cryptoCompare api
    url = "https://min-api.cryptocompare.com/data/histominute"
    # querry str
    xQuerryStr =  {"fsym":fSymbol,"tsym":tSymbol,"limit":"60","aggregate":"1",  
               "e":xExchange,"toTs":timeStamp}
    yQuerryStr =  {"fsym":fSymbol,"tsym":tSymbol,"limit":"60","aggregate":"1",  
               "e":yExchange,"toTs":timeStamp}
    # request response
    try:
        xResponse = requests.request("GET", url, params=xQuerryStr)
        yResponse = requests.request("GET", url, params=yQuerryStr)
        if xResponse.status_code == requests.codes.ok and yResponse.status_code == requests.codes.ok:
            # get xDataFrame, yDataFrame
            xDataFrame = pd.DataFrame(xResponse.json()["Data"])
            yDataFrame = pd.DataFrame(yResponse.json()["Data"])
            # calc measurements
            measurements = calcMeasurements(xDataFrame, yDataFrame, xFee, yFee)
            chance = 0
            profit = 0
            volume = 0
            candy = 0
            for index in range(len(measurements)):
                if measurements.loc[index,"profit"] > 0:
                    if measurements.loc[index,"difference"] != 0 and candy == 0:
                        if measurements.loc[index,"difference"] > 0:
                            candy = 1
                        else:
                            candy = -1
                        chance = 1
                        profit = measurements.loc[index,"profit"]
                        volume = measurements.loc[index,"volume"]
                    if measurements.loc[index,"difference"] > 0 and candy < 0:
                        candy = 1
                        chance = chance + 1
                        profit = profit + measurements.loc[index,"profit"]
                        volume = volume + measurements.loc[index,"volume"]
                    if measurements.loc[index,"difference"] < 0 and candy > 0:
                        candy = -1
                        chance = chance + 1
                        profit = profit + measurements.loc[index,"profit"]
                        volume = volume + measurements.loc[index,"volume"]
            return [chance, profit, volume]
        else:
            return [-1, -1, -1]
    except requests.exceptions.RequestException as e:
        print(e)
        sys.exit(1)

def calcMeasurements(xDataFrame, yDataFrame, xFee, yFee):
    # checkout dataFrame
    xDataFrame.set_index(["time"], inplace=True)
    yDataFrame.set_index(["time"], inplace=True)
    # get the intersection
    ixs = xDataFrame.index.intersection(yDataFrame.index)
    # get x and y sample
    xDataSample = xDataFrame.loc[ixs]
    yDataSample = yDataFrame.loc[ixs]
    xDataSample.reset_index(drop=False, inplace=True)
    yDataSample.reset_index(drop=False, inplace=True)
    # calc measurements
    measurements = pd.DataFrame(columns=("time", "difference", "profit", "volume"))
    measurements["time"] = pd.Series(ixs)
    measurements["difference"] = pd.Series(xDataSample["close"] - yDataSample["close"])
    measurements["profit"] = pd.Series(np.abs(xDataSample["close"] - yDataSample["close"]) 
                                - (xDataSample["close"] * xFee + yDataSample["close"] * yFee))
    measurements["volume"] = pd.Series(np.minimum(xDataSample["volumeto"], yDataSample["volumeto"]))
    return measurements

print("done")

done


In [ ]:
 # calc arbitrage opportunity in Okex and Binance

timeStr = "2018-11-4 18:00:00"
okexBinanceCoinFile = "../data/cryptocompare/cryptocompare.exchange.okexBinanceCoin.json"

# Reading data back
with open(okexBinanceCoinFile, 'r') as f:
    okexBinanceCoin = json.load(f)

# Calc timeStamp
timeStamp = int(time.mktime(time.strptime(timeStr, "%Y-%m-%d %H:%M:%S")))

# chance, profit, volume = calcHourArbitrage("DNT", "ETH", "OKEX", 0.002, "Binance", 0.001, timeStamp)

for fsym in okexBinanceCoin:
    for tsym in okexBinanceCoin[fsym]:
        [chance, profit, volume] = calcHourArbitrage(fsym, tsym, "OKEX", 0.002, "Binance", 0.001, timeStamp)        
        print(fsym,'vs',tsym)
        print("   chance =", chance)
        print("   profit =", profit)
        print("   volume =", volume)

INS vs BTC
   chance = 8
   profit = 1.8084600000000198e-06
   volume = 0.16395699999999996
INS vs ETH
   chance = 12
   profit = 0.0002079060000000007
   volume = 0.42716361
POE vs BTC
   chance = 1
   profit = 3.799999999999903e-09
   volume = 0.0
POE vs ETH
   chance = 1
   profit = 4.426099999999938e-07
   volume = 0.0
DGD vs BTC
   chance = 2
   profit = 4.4306999999999744e-05
   volume = 0.0
DGD vs ETH
   chance = 2
   profit = 0.0007722000000000019
   volume = 0.0
EVX vs BTC
   chance = 1
   profit = 6.45e-07
   volume = 0.0
EVX vs ETH
   chance = 1
   profit = 4.5091000000000094e-05
   volume = 0.0
GTO vs BTC
   chance = 1
   profit = 4.880999999999923e-08
   volume = 1.05
GTO vs ETH
   chance = 1
   profit = 1.6115999999999546e-06
   volume = 11.93
OST vs BTC
   chance = 3
   profit = 3.904000000000233e-08
   volume = 0.0
OST vs ETH
   chance = 2
   profit = 5.7147999999999855e-06
   volume = 0.4514
BTG vs BTC
   chance = 4
   profit = 2.881399999999974e-05
   volume = 0.03199

In [ ]:
 # calc arbitrage opportunity in Okex and Huobi

timeStr = "2018-11-4 18:00:00"
okexHuobiCoinFile = "../data/cryptocompare/cryptocompare.exchange.okexHuobiCoin.json"

# Reading data back
with open(okexHuobiCoinFile, 'r') as f:
    okexHuobiCoin = json.load(f)

# Calc timeStamp
timeStamp = int(time.mktime(time.strptime(timeStr, "%Y-%m-%d %H:%M:%S")))

# chance, profit, volume = calcHourArbitrage("DNT", "ETH", "OKEX", 0.002, "Huobi", 0.001, timeStamp)

for fsym in okexHuobiCoin:
    for tsym in okexHuobiCoin[fsym]:
        [chance, profit, volume] = calcHourArbitrage(fsym, tsym, "OKEX", 0.002, "HuobiPro", 0.001, timeStamp)        
        print(fsym,'vs',tsym)
        print("   chance =", chance)
        print("   profit =", profit)
        print("   volume =", volume)

In [ ]:
 # calc arbitrage opportunity in Okex and Gate

timeStr = "2018-11-4 18:00:00"
okexGateCoinFile = "../data/cryptocompare/cryptocompare.exchange.okexGateCoin.json"

# Reading data back
with open(okexGateCoinFile, 'r') as f:
    okexGateCoin = json.load(f)

# Calc timeStamp
timeStamp = int(time.mktime(time.strptime(timeStr, "%Y-%m-%d %H:%M:%S")))

# chance, profit, volume = calcHourArbitrage("DNT", "ETH", "OKEX", 0.002, "Gate", 0.001, timeStamp)

for fsym in okexGateCoin:
    for tsym in okexGateCoin[fsym]:
        [chance, profit, volume] = calcHourArbitrage(fsym, tsym, "OKEX", 0.002, "Gateio", 0.001, timeStamp)        
        print(fsym,'vs',tsym)
        print("   chance =", chance)
        print("   profit =", profit)
        print("   volume =", volume)

In [ ]:
 # calc arbitrage opportunity in binance and Huobi

timeStr = "2018-11-4 18:00:00"
binanceHuobiCoinFile = "../data/cryptocompare/cryptocompare.exchange.binanceHuobiCoin.json"

# Reading data back
with open(binanceHuobiCoinFile, 'r') as f:
    binanceHuobiCoin = json.load(f)

# Calc timeStamp
timeStamp = int(time.mktime(time.strptime(timeStr, "%Y-%m-%d %H:%M:%S")))

# chance, profit, volume = calcHourArbitrage("DNT", "ETH", "binance", 0.002, "Huobi", 0.001, timeStamp)

for fsym in binanceHuobiCoin:
    for tsym in binanceHuobiCoin[fsym]:
        [chance, profit, volume] = calcHourArbitrage(fsym, tsym, "Binance", 0.002, "HuobiPro", 0.001, timeStamp)        
        print(fsym,'vs',tsym)
        print("   chance =", chance)
        print("   profit =", profit)
        print("   volume =", volume)

In [ ]:
 # calc arbitrage opportunity in binance and Gate

timeStr = "2018-11-4 18:00:00"
binanceGateCoinFile = "../data/cryptocompare/cryptocompare.exchange.binanceGateCoin.json"

# Reading data back
with open(binanceGateCoinFile, 'r') as f:
    binanceGateCoin = json.load(f)

# Calc timeStamp
timeStamp = int(time.mktime(time.strptime(timeStr, "%Y-%m-%d %H:%M:%S")))

# chance, profit, volume = calcHourArbitrage("DNT", "ETH", "binance", 0.002, "Gate", 0.001, timeStamp)

for fsym in binanceGateCoin:
    for tsym in binanceGateCoin[fsym]:
        [chance, profit, volume] = calcHourArbitrage(fsym, tsym, "Binance", 0.002, "Gateio", 0.001, timeStamp)        
        print(fsym,'vs',tsym)
        print("   chance =", chance)
        print("   profit =", profit)
        print("   volume =", volume)

In [ ]:
 # calc arbitrage opportunity in huobi and Gate

timeStr = "2018-11-4 18:00:00"
huobiGateCoinFile = "../data/cryptocompare/cryptocompare.exchange.huobiGateCoin.json"

# Reading data back
with open(huobiGateCoinFile, 'r') as f:
    huobiGateCoin = json.load(f)

# Calc timeStamp
timeStamp = int(time.mktime(time.strptime(timeStr, "%Y-%m-%d %H:%M:%S")))

# chance, profit, volume = calcHourArbitrage("DNT", "ETH", "huobi", 0.002, "Gate", 0.001, timeStamp)

for fsym in huobiGateCoin:
    for tsym in huobiGateCoin[fsym]:
        [chance, profit, volume] = calcHourArbitrage(fsym, tsym, "HuobiPro", 0.002, "Gateio", 0.001, timeStamp)        
        print(fsym,'vs',tsym)
        print("   chance =", chance)
        print("   profit =", profit)
        print("   volume =", volume)